# Model Selection Demo

Model evaluation and model selection are two different concepts, representing two different stages:
- Model selection: Select the best model from a certain model space according to a set of model representations of different complexity;
- Model evaluation: After selecting a (best) model, evaluate its performance such as prediction error against new data.

We adopt the dataset from Genome-wide cell-free DNA fragmentation in patients with cancer as the example.

## Some Package Import and Data Preparation

In [239]:
import pandas as pd
import numpy as np
import warnings


# from numpy import *
from sklearn import *

warnings.filterwarnings('ignore')

xl = pd.ExcelFile('data2.xlsx')
xl.sheet_names # we'll take 7th
dfs = {sheet: xl.parse(sheet) for sheet in xl.sheet_names}
data1 = dfs['7']
data2 = dfs['1'].loc[:,['Patient','Age at Diagnosis']].drop([554]).drop_duplicates()
data3 = pd.read_csv('data1.csv')
combined_data = data1.set_index('Patient').join(data2.set_index('Patient')).join(data3.set_index('Patient'))
combined_data['label'] = (combined_data['Patient Type'] == 'Healthy').astype(int)
combined_data = combined_data.drop(['Patient Type'],axis=1)
print('The number of samples and features are %d and %d, respectively'%(combined_data.shape[0],combined_data.shape[1]))

x = combined_data.iloc[:, 0:44]
x[isnan(x)] = 0
y=combined_data.iloc[:,44]

The number of samples and features are 423 and 45, respectively


## Resample Methods: Nested k-fold cross validation

- Select different type of models.

In [103]:
##---  Model selection demo based on from support vector machine, random forest, k-nearest neighbors, using nested k-fold cross-validation ---##

from sklearn.model_selection import KFold

##---  Prepare the Outer Loop cross-validation procedure ---##
OutKF = KFold(n_splits=5,shuffle=True,random_state=920)
##---   If we want to make sure that the proportion of samples in the training and test set is the same as in the original dataset, we should adopt 'StratifiedKFold' ---##
# OutKF = StratifiedKFold(n_outer,shuffle=True, random_state=920+i)

scores = []
for train_index,test_index in OutKF.split(x,y):
    ##---  Seperate traing set and test set ---##
    x_train, x_test = x.iloc[train_index][:], x.iloc[test_index][:]
    y_train, y_test = y.iloc[train_index][:], y.iloc[test_index][:]

    ##---  Prepare the Inner Loop cross-validation procedure ---##
    valscores0 = []
    valscores1 = []
    valscores2 = []
    InnKF = KFold(n_splits=5,shuffle=True,random_state=920)
    for subtrain_index,valid_index in InnKF.split(x_train,y_train):
        #---  Seperate subtraing set and validation set ---#
        x_subtrain, x_valid = x_train.iloc[subtrain_index][:], x_train.iloc[valid_index][:]
        y_subtrain, y_valid = y_train.iloc[subtrain_index][:], y_train.iloc[valid_index][:]
        #---  creat and train the model ---#
        clf0 = svm.SVC(kernel = 'rbf', gamma='auto', probability=True,random_state=920).fit(x_subtrain,y_subtrain)
        clf1 = ensemble.RandomForestClassifier(random_state=920).fit(x_subtrain,y_subtrain)
        clf2 = neighbors.KNeighborsClassifier().fit(x_subtrain,y_subtrain)
        #---  Make the prediction ---#
        y_valpred0 = clf0.predict(x_valid)
        y_valpred1 = clf1.predict(x_valid)
        y_valpred2 = clf2.predict(x_valid)
        #---  Evaluate model on the specific validation set ---#
        valscore0 = metrics.accuracy_score(y_valid,y_valpred0)
        valscores0.append(valscore0) 
        valscore1 = metrics.accuracy_score(y_valid,y_valpred1)
        valscores1.append(valscore1)
        valscore2 = metrics.accuracy_score(y_valid,y_valpred2)
        valscores2.append(valscore2)
    valscore = [mean(valscore0),mean(valscore1),mean(valscore2)]
    max_index = valscore.index(max(valscore))
        
    ##---  Seclet the best model and evaluate model on the specific test set ---##
    # clf0 = svm.SVC(kernel = 'rbf', gamma='auto', probability=True).fit(x_train,y_train)
    # clf1 = ensemble.RandomForestClassifier().fit(x_train,y_train)
    # clf2 = neighbors.KNeighborsClassifier().fit(x_train,y_train)
    if max_index == 0:
        clf = svm.SVC(kernel = 'rbf', gamma='auto', probability=True).fit(x_train,y_train)
        print('Classifier with higest accuracy on validation set is SVM.')
    else:
        if max_index == 1:
            clf = ensemble.RandomForestClassifier().fit(x_train,y_train)
            print('Classifier with higest accuracy on validation set is RandomForest.')
        else:
            clf = neighbors.KNeighborsClassifier().fit(x_train,y_train)
            print('Classifier with higest accuracy on validation set is KNN.')
   
    ##---  Evaluate model on the specific test set ---##
    y_pred = clf.predict(x_test)
    score = metrics.accuracy_score(y_test, y_pred)
    scores.append(score) 

##---  Report performance on test set---##
print('Accuracy in each split:',scores)
print('Mean and  standard deviation of accuracy: %.2f (%.2f)' % (mean(scores),std(scores)))

Classifier with higest accuracy on validation set is RandomForest.
Classifier with higest accuracy on validation set is RandomForest.
Classifier with higest accuracy on validation set is SVM.
Classifier with higest accuracy on validation set is SVM.
Classifier with higest accuracy on validation set is SVM.
Accuracy: 0.85 (0.03)


- Select different hyperparameters of models.

In [132]:
##---  Model selection demo based on different number of neighbors for KNN, using nested k-fold cross-validation ---##

from sklearn.model_selection import KFold

n_outer = 5 # number of splits for outer loop
n_inner = 5 # number of splits for inner loop
range_n = [1, 10**2] # list, float, range of parameter n
step_n =10 # step of candidate parameters

parameters = {'n_neighbors':range(range_n[0],range_n[1]+2,step_n)}

##---  Prepare the Outer Loop cross-validation procedure ---##
OutKF = KFold(n_splits=5,shuffle=True,random_state=920)
##---   If we want to make sure that the proportion of samples in the training and test set is the same as in the original dataset, we should adopt 'StratifiedKFold' ---##
# OutKF = StratifiedKFold(n_outer,shuffle=True, random_state=920+i)

scores = []
for train_index,test_index in OutKF.split(x,y):
    ##---  Seperate traing set and test set ---##
    x_train, x_test = x.iloc[train_index][:], x.iloc[test_index][:]
    y_train, y_test = y.iloc[train_index][:], y.iloc[test_index][:]

    ##---  Implement the Inner Loop cross-validation and parameter exploring procedure using grid search ---##
    estimator = neighbors.KNeighborsClassifier()
    clf = model_selection.GridSearchCV(estimator,parameters,cv=n_inner,verbose=0,scoring='accuracy').fit(x_train, y_train)
    
#     print('Higest accuracy and the corresponding parameter is : %.2f (%d).' %(clf.best_score_,clf.best_params_['n_neighbors']))

    ##---  Evaluate model on the specific test set ---##
    y_pred = clf.predict(x_test)
    score = metrics.accuracy_score(y_test, y_pred)
    scores.append(score) 

##---  Report performance on test set---##
print('Accuracy in each split:',scores)
print('Mean and  standard deviation of accuracy: %.2f (%.2f)' % (mean(scores),std(scores)))

Higest accuracy and the corresponding parameter is : 0.72 (11).
Higest accuracy and the corresponding parameter is : 0.74 (1).
Higest accuracy and the corresponding parameter is : 0.70 (11).
Higest accuracy and the corresponding parameter is : 0.72 (11).
Higest accuracy and the corresponding parameter is : 0.71 (11).
Accuracy in each split: [0.8470588235294118, 0.7411764705882353, 0.788235294117647, 0.8214285714285714, 0.7142857142857143]
Mean and  standard deviation of accuracy: 0.78 (0.05)


## Analytical Measures

- Akaike Information Criterion (AIC)

In [288]:
##---  Model selection demo based on AIC for LASSO nested in the model evaluation method of bootstrap ---##

from mlxtend.evaluate import bootstrap_point632_score,BootstrapOutOfBag

##---  Compute number of paramters for the linear model ---##
def compute_num_parameters(coef):
    num_para = np.sum((np.abs(coef) > np.finfo(coef.dtype).eps)!=0)
    return num_para

##---  Compute AIC for SVM with linear kernel ---##
def compute_aic(n, mse, num_params):
    AIC = n* n * log(mse) + 2 * num_params
    return AIC

##---  Prepare the bootstrap procedure ---##
n_outer = 3 # number of splits/repeats for outer loop (Model evaluation)
oob = BootstrapOutOfBag(n_splits=n_outer,random_seed=920)

scores = []
i = 0
for train_index, test_index in oob.split(x,y):
    i+=1
    print('- '*10)
    print('In %dth iteration:'%(i))
    print('- '*10)
    ##---  Seperate traing set and test set ---##
    x_train, x_test = x.iloc[train_index][:], x.iloc[test_index][:]
    y_train, y_test = y.iloc[train_index][:],y.iloc[test_index][:]
    ##---  creat and train the model with different parameters---##
    clf0 = linear_model.Lasso(alpha=0.1).fit(x_train,y_train)
    clf1 = linear_model.Lasso(alpha=0.2).fit(x_train,y_train)
    clf2 = linear_model.Lasso(alpha=0.3).fit(x_train,y_train)
    ##---  Make the prediction ---##
    y_pred0 = clf0.predict(x_train)
    y_pred1 = clf1.predict(x_train)
    y_pred2 = clf2.predict(x_train)
    
    ##---  Compute AIC on the specific training set ---##
    # number of parameters
    num_params0 = compute_num_parameters(clf0.coef_) + 1
    print('Number of parameters for clf0: %d' % (num_params0))
    num_params1 = compute_num_parameters(clf1.coef_) + 1
    print('Number of parameters for clf1: %d' % (num_params1))
    num_params2 = compute_num_parameters(clf2.coef_) + 1
    print('Number of parameters for clf2: %d' % (num_params2))
    # maximum likelihood
    mse0 = metrics.mean_squared_error(y_train, y_pred0)
    print('Mse for clf0: %f' % mse0)
    mse1 = metrics.mean_squared_error(y_train, y_pred1)
    print('Mse for clf1: %f' % mse1)
    mse2 = metrics.mean_squared_error(y_train, y_pred2)
    print('Mse for clf2: %f' % mse2)
    # AIC
    AIC = [compute_aic(len(y_train), mse0, num_params0), compute_aic(len(y_train), mse1, num_params1), 
            compute_aic(len(y_train), mse2, num_params2)]
    print('AIC:',AIC)
    
    ##---  Seclet the best model and evaluate model on the specific test set ---##
    min_index = AIC.index(min(AIC))
    print('The best model is model %d.' % min_index)
    if min_index == 0:
        y_pred = int64(clf0.predict(x_test)>0.5)
    else:
        if min_index == 1:
            y_pred = int64(clf1.predict(x_test)>0.5)
        else:
            y_pred = int64(clf2.predict(x_test)>0.5)
   
    ##---  Evaluate model on the specific test set ---##
    score = metrics.accuracy_score(y_test, y_pred)
    scores.append(score)
    

# ##---  Report performance ---##
print('- '*20)
print('Final model：')
print('Accuracy: %.2f (%.2f)' % (mean(scores), std(scores)), scores)
print('- '*20)

- - - - - - - - - - 
In 1th iteration:
- - - - - - - - - - 
Number of parameters for clf0: 13
Number of parameters for clf1: 9
Number of parameters for clf2: 3
Mse for clf0: 0.158660
Mse for clf1: 0.182019
Mse for clf2: 0.208360
AIC: [-329380.3199936178, -304813.523050753, -280641.97077876364]
The best model is model 0.
- - - - - - - - - - 
In 2th iteration:
- - - - - - - - - - 
Number of parameters for clf0: 13
Number of parameters for clf1: 8
Number of parameters for clf2: 5
Mse for clf0: 0.152828
Mse for clf1: 0.180776
Mse for clf2: 0.208004
AIC: [-336081.98603828734, -306041.23911747476, -280944.15397766704]
The best model is model 0.
- - - - - - - - - - 
In 3th iteration:
- - - - - - - - - - 
Number of parameters for clf0: 12
Number of parameters for clf1: 8
Number of parameters for clf2: 4
Mse for clf0: 0.164440
Mse for clf1: 0.191215
Mse for clf2: 0.216330
AIC: [-322980.6594710439, -295996.28212320723, -273923.59108677466]
The best model is model 0.
- - - - - - - - - - - - - - -

- Bayesian Information Criterion (BIC)

In [286]:
##---  Model selection demo based on AIC for LASSO nested in the model evaluation method of bootstrap ---##

from mlxtend.evaluate import bootstrap_point632_score,BootstrapOutOfBag

##---  Compute number of paramters for the linear model ---##
def compute_num_parameters(coef):
    num_para = np.sum((np.abs(coef) > np.finfo(coef.dtype).eps)!=0)
    return num_para

##---  Compute BIC ---##
def compute_bic(n, mse, num_params):
    BIC = n * n * log(mse) + num_params * log(n)
    return BIC

##---  Prepare the bootstrap procedure ---##
n_outer = 3 # number of splits/repeats for outer loop (Model evaluation)
oob = BootstrapOutOfBag(n_splits=n_outer,random_seed=920)

scores = []
i = 0
for train_index, test_index in oob.split(x,y):
    i+=1
    print('- '*10)
    print('In %dth iteration:'%(i))
    print('- '*10)
    ##---  Seperate traing set and test set ---##
    x_train, x_test = x.iloc[train_index][:], x.iloc[test_index][:]
    y_train, y_test = y.iloc[train_index][:],y.iloc[test_index][:]
    ##---  creat and train the model with different parameters---##
    clf0 = linear_model.Lasso(alpha=0.01).fit(x_train,y_train)
    clf1 = linear_model.Lasso(alpha=0.1).fit(x_train,y_train)
    clf2 = linear_model.Lasso(alpha=0.2).fit(x_train,y_train)
    ##---  Make the prediction ---##
    y_pred0 = clf0.predict(x_train)
    y_pred1 = clf1.predict(x_train)
    y_pred2 = clf2.predict(x_train)
    
    ##---  Compute AIC on the specific training set ---##
    # number of parameters
    num_params0 = compute_num_parameters(clf0.coef_) + 1
    print('Number of parameters for clf0: %d' % (num_params0))
    num_params1 = compute_num_parameters(clf1.coef_) + 1
    print('Number of parameters for clf1: %d' % (num_params1))
    num_params2 = compute_num_parameters(clf2.coef_) + 1
    print('Number of parameters for clf2: %d' % (num_params2))
    # maximum likelihood
    mse0 = metrics.mean_squared_error(y_train, y_pred0)
    print('Mse for clf0: %f' % mse0)
    mse1 = metrics.mean_squared_error(y_train, y_pred1)
    print('Mse for clf1: %f' % mse1)
    mse2 = metrics.mean_squared_error(y_train, y_pred2)
    print('Mse for clf2: %f' % mse2)
    # BIC
    BIC = [compute_bic(len(y_train), mse0, num_params0), compute_bic(len(y_train), mse1, num_params1), 
            compute_bic(len(y_train), mse2, num_params2)]
    print('BIC:',BIC)
    
    ##---  Seclet the best model and evaluate model on the specific test set ---##
    min_index = BIC.index(min(BIC))
    print('The best model is model %d.' % min_index)
    if min_index == 0:
        y_pred = int64(clf0.predict(x_test)>0.5)
    else:
        if min_index == 1:
            y_pred = int64(clf1.predict(x_test)>0.5)
        else:
            y_pred = int64(clf2.predict(x_test)>0.5)
   
    ##---  Evaluate model on the specific test set ---##
    score = metrics.accuracy_score(y_test, y_pred)
    scores.append(score)
    

# ##---  Report performance ---##
print('- '*20)
print('Final model：')
print('Accuracy: %.2f (%.2f)' % (mean(scores), std(scores)), scores)
print('- '*20)

- - - - - - - - - - 
In 1th iteration:
- - - - - - - - - - 
Number of parameters for clf0: 31
Number of parameters for clf1: 13
Number of parameters for clf2: 9
Mse for clf0: 0.120383
Mse for clf1: 0.158660
Mse for clf2: 0.182019
BIC: [-378619.3264831722, -329327.7041552902, -304777.09670114156]
The best model is model 0.
- - - - - - - - - - 
In 2th iteration:
- - - - - - - - - - 
Number of parameters for clf0: 31
Number of parameters for clf1: 13
Number of parameters for clf2: 8
Mse for clf0: 0.118425
Mse for clf1: 0.152828
Mse for clf2: 0.180776
BIC: [-381553.040246516, -336029.3701999597, -306008.8601400424]
The best model is model 0.
- - - - - - - - - - 
In 3th iteration:
- - - - - - - - - - 
Number of parameters for clf0: 33
Number of parameters for clf1: 12
Number of parameters for clf2: 8
Mse for clf0: 0.119737
Mse for clf1: 0.164440
Mse for clf2: 0.191215
BIC: [-379569.4989552182, -322932.09100489534, -295963.90314577485]
The best model is model 0.
- - - - - - - - - - - - - - -

- Minimum  Description  Length  (MDL)

In [ ]:
##---  Model selection demo based on AIC for SVM nested in the model evaluation method of bootstrap ---##

##---  Compute AIC for SVM with linear kernel ---##
def compute_aic(n, mse, num_params):
    AIC = n * log(mse) + 2 * num_params
    return AIC
from mlxtend.evaluate import bootstrap_point632_score,BootstrapOutOfBag

n_outer = 2 # number of splits/repeats for outer loop (Model evaluation)
rangeC = [10**-2,5] # list, float, range of parameter C,eg.[10**-2, 10**2]
rangeGamma = [10**-2,1] # list, float, range of parameter gamma,eg.[10**-6, 10**1]
num_C = 3 # number of cadidate values of parameter C for SVM, to tune the interval of parameter
num_gamma = 3 # number of cadidate values of parameter gamma for SVM, to tune the interval of parameter
parameters = {'C':linspace(rangeC[0],rangeC[1],num_C),'gamma':linspace(rangeGamma[0],rangeGamma[1],num_gamma)}


##---  Prepare the bootstrap procedure ---##
oob = BootstrapOutOfBag(n_splits=n_outer,random_seed=920)

scores = []
for train_index, test_index in oob.split(x,y):
    print('-'*20)
    print('In %dth iteration:'%(oob.get_n_splits()))
    ##---  Seperate traing set and test set ---##
    x_train, x_test = x.iloc[train_index][:], x.iloc[test_index][:]
    y_train, y_test = y.iloc[train_index][:],y.iloc[test_index][:]
    ##---  creat and train the model with different parameters---##
    clf0 = svm.SVC(kernel='linear',C=parameters['C'][0],gamma=parameters['gamma'][0]).fit(x_train,y_train)
    clf1 = svm.SVC(kernel='linear',C=parameters['C'][1],gamma=parameters['gamma'][1]).fit(x_train,y_train)
    clf2 = svm.SVC(kernel='linear',C=parameters['C'][2],gamma=parameters['gamma'][2]).fit(x_train,y_train)
#     clf0 = linear_model.RidgeClassifier(alpha=0).fit(x_train,y_train)
#     clf1 = linear_model.RidgeClassifier(alpha=1).fit(x_train,y_train)
#     clf2 = linear_model.RidgeClassifier(alpha=300).fit(x_train,y_train)
#     clf0 = linear_model.Lasso(alpha=0).fit(x_train,y_train)
#     clf1 = linear_model.Lasso(alpha=0.1).fit(x_train,y_train)
#     clf2 = linear_model.Lasso(alpha=1).fit(x_train,y_train)
    ##---  Make the prediction ---##
    y_pred0 = clf0.predict(x_train)
    y_pred1 = clf1.predict(x_train)
    y_pred2 = clf2.predict(x_train)
    print(clf0.coef_[0])
    print(clf1.coef_[0])
    print(clf2.coef_[0])
    
    ##---  Compute AIC on the specific training set ---##
    # number of parameters
    num_params0 = len(clf0.coef_[0]) + 1
    print('Number of parameters for clf0: %d' % (num_params0))
    num_params1 = len(clf1.coef_[0]) + 1
    print('Number of parameters for clf1: %d' % (num_params1))
    num_params2 = len(clf2.coef_[0]) + 1
    print('Number of parameters for clf2: %d' % (num_params2))

    
#     ##---  Compute AIC on the specific training set ---##
#     # number of parameters
#     num_params0 = len(clf0.coef_) + 1
#     print('Number of parameters for clf0: %d' % (num_params0))
#     num_params1 = len(clf1.coef_) + 1
#     print('Number of parameters for clf1: %d' % (num_params1))
#     num_params2 = len(clf2.coef_) + 1
#     print('Number of parameters for clf2: %d' % (num_params2))
    # maximum likelihood
    mse0 = metrics.mean_squared_error(y_train, y_pred0)
    print('Mse for clf0: %f' %mse0)
    mse1 = metrics.mean_squared_error(y_train, y_pred1)
    print('Mse for clf1: %f' %mse1)
    mse2 = metrics.mean_squared_error(y_train, y_pred2)
    print('Mse for clf2: %f' %mse2)
    
    
    
    max_index = valscore.index(max(valscore))
    ##---  Evaluate model on the specific test set ---##
#     y_pred = clf1.predict(x_test)
    y_pred = int64(clf0.predict(x_test))
    score = metrics.accuracy_score(y_test, y_pred)
    scores.append(score)




# ##---  Report performance ---##
print('Accuracy: %.2f (%.2f)' % (mean(scores), std(scores)))
print(scores)